<a href="https://colab.research.google.com/github/LabeebHameed/09/blob/main/Pulling_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/tweepy/tweepy.git

  Cloning https://github.com/tweepy/tweepy.git to /tmp/pip-req-build-69kkmsx0
  Running command git clone --filter=blob:none --quiet https://github.com/tweepy/tweepy.git /tmp/pip-req-build-69kkmsx0
  Resolved https://github.com/tweepy/tweepy.git to commit db28c0e84826485755eb7fcef0c30f75395dff5f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
pip install --upgrade tweepy

In [13]:
import tweepy
import pandas as pd

# Replace this with your Bearer Token
bearer_token = "AAAAAAAAAAAAAAAAAAAAAB0I0QEAAAAAYFknIYAMGK2lIugbPefsRmY%2Fb4M%3DxlUXyR6pVVorftoEqRtpu7bNb6xSkke1hA1C14zxXYnV5xspco"

# Set up client with bearer token
client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)

# Search query (Twitter API v2 syntax is similar)
username = "elonmusk"  # Change this to whoever you want

# Step 1: Get user ID
user = client.get_user(username=username)
user_id = user.data.id

print(f"Fetching data for @{username} (User ID: {user_id})...\n")

# Step 2: Get all followers
print("Fetching followers...")
followers = []
for response in tweepy.Paginator(
    client.get_users_followers,
    id=user_id,
    max_results=1000,  # Max 1000 for Elevated access, 100 for Essential
    user_fields=["username", "name", "public_metrics", "verified", "created_at"]
):
    if response.data:
        followers.extend(response.data)

print(f"Total Followers Fetched: {len(followers)}")

# Step 3: Get all following
print("Fetching following...")
following = []
for response in tweepy.Paginator(
    client.get_users_following,
    id=user_id,
    max_results=1000,
    user_fields=["username", "name", "public_metrics", "verified", "created_at"]
):
    if response.data:
        following.extend(response.data)

print(f"Total Following Fetched: {len(following)}")

# Step 4: Convert to DataFrame
def users_to_df(users_list):
    return pd.DataFrame([
        {
            "Username": u.username,
            "Name": u.name,
            "Followers": u.public_metrics["followers_count"],
            "Following": u.public_metrics["following_count"],
            "Verified": u.verified,
            "Created At": u.created_at
        }
        for u in users_list
    ])

followers_df = users_to_df(followers)
following_df = users_to_df(following)

# Step 5: Save to CSV (optional)
followers_df.to_csv(f"{username}_followers.csv", index=False)
following_df.to_csv(f"{username}_following.csv", index=False)

print("\n✅ Data saved to CSV files.")



Username: Labeeb_Hameed
Name: Labeeb Hameed
Created at: 2019-02-08 12:12:02+00:00
Followers: 3
Verified: False
ID: 1093844904281665536


In [ ]:
tweets_df

In [ ]:
tweets[0]

In [ ]:
from google.colab import files
tweets_df.to_csv('tweets.csv')
files.download('tweets.csv')